<a href="https://colab.research.google.com/github/reyllama/TTIO/blob/master/08_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
data = pd.read_csv("/content/drive/My Drive/core_stocks.csv", dtype={'stock_id': str})
data['stock_id'] = data['stock_id'].astype(str)
data.shape[0]//979

1586

In [8]:
df = pd.read_csv('/content/drive/My Drive/corr1.csv')
df = df.set_index(keys='stock_id')
df.columns = df.columns.astype(str)
df.index = df.index.astype(str)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def get_embeddings(corr_df=None, n_epochs=100, emb_dim=32, lr=0.01, silent=False):
  dtype = torch.FloatTensor
  epochs = n_epochs
  embeddings = Variable(torch.randn(emb_dim, len(corr_df)).type(dtype), requires_grad=True) # Initialize with Standard Gaussian
  optimizer = optim.Adam([embeddings], lr=lr)
  mean = np.abs(corr_df).mean().mean()
  coef = 1/mean # Hyper Parameter
  loss_track = []

  for epoch in range(epochs):
    loss = 0
    for i in range(len(corr_df)):
      v1 = embeddings[:, i].view(emb_dim, -1)
      dist = torch.norm(v1 - embeddings, dim=0).view(len(corr_df), 1)
      corrs = torch.from_numpy(np.abs(coef*np.array(corr_df.iloc[:, i]))-1).view(1, len(corr_df)).type(torch.FloatTensor)
    #   corrs = torch.from_numpy(np.array(df.iloc[:, i])).view(1, len(df)).type(torch.FloatTensor) # rho 교수님 버전
      loss += torch.mm(corrs, dist)/2

    #   v1 = embeddings[:, i].view(-1, emb_dim)
    #   mm = torch.mm(v1, embeddings)
    #   mm[:, i] = torch.zeros(1) # 자기 자신과의 dot product은 0으로 만들어줌. 그렇지 않으면 각각이 다 0 벡터가 될 우려
    #   corrs = torch.from_numpy(coef*np.array(df.iloc[:, i])).view(len(df), 1).type(torch.FloatTensor)
    #   loss += torch.mm(mm, corrs)

    loss_track.append(loss.item())
    if not silent:
        if (epoch+1) % 20 == 0:
            print("{0}th epoch in process".format(epoch+1))
            print('running loss: {}'.format(loss.item()))
            print()

    optimizer.zero_grad()
    loss.backward(retain_graph=False)
    optimizer.step()

    with torch.no_grad():
      for element in embeddings:
        element.clamp_(0,1)

  return embeddings

In [5]:
def data_preparer(n=1, n_epochs=120, lr=0.05, silent=False, corr=True):
    if corr:
        path = "/content/drive/My Drive/corr{0}.csv".format(n)
    if not corr:
        path = "/content/drive/My Drive/cov{0}.csv".format(n)
    df = pd.read_csv(path)
    df = df.set_index(keys='stock_id')
    df.columns = df.columns.astype(str)
    df.index = df.index.astype(str)

    if n==1:
        st, ed = 1, 121
    elif n==2:
        st, ed = 62, 186
    elif n==3:
        st, ed=124,248
    elif n==4:
        st, ed=186,310
    elif n==5:
        st, ed=248, 368
    elif n==6:
        st, ed=310, 432
    elif n==7:
        st, ed=368,489
    elif n==8:
        st, ed=432, 550
    else:
        print("Too Big A Number")
        return None

    embeddings = get_embeddings(corr_df=df, n_epochs=n_epochs, lr=lr, silent=silent)

    return df, embeddings, st, ed

In [6]:
!pip install ta
import ta

def bias(close=None, m=7):
  return close - close.rolling(m).mean()

def preprocessing(data=data, start=125, end=125+62, corr_df = None):

  data['stock_id'] = data['stock_id'].astype(str)
  data['year'] = data['date'].apply(lambda x: int(x[0:4]))
  data['month'] = data['date'].apply(lambda x: int(x[5:7]))

  data = data[(data['date_index']<=end)&(data['date_index']>=start)]
  data = data[data['stock_id'].isin(df.columns)]

  data['KLength'] = data['closing_price'] - data['opening_price']
  data['KUpperLength'] = data['highest_price'] - data[['opening_price', 'closing_price']].max(axis=1)
  data['temp'] = data['opening_price']-data['lowest_price']
  data['KLowerLength'] = data[['closing_price', 'temp']].min(axis=1)
  data['EMA'] = ta.trend.ema_indicator(close=data['closing_price'])
  data['MACD'] = ta.trend.macd(close=data['closing_price'])
  data['ROC'] = ta.momentum.roc(close=data['closing_price'])
  data['BIAS'] = bias(data['closing_price'])
  data['return'] = data['closing_price'].diff()
  data = data.drop(columns=['temp'])

  
  

  return data

  Created wheel for ta: filename=ta-0.5.25-cp36-none-any.whl size=24880 sha256=d14d4bd1d81ebb05c48e00fd2ee5c678020c38da094ac3787719e9727c563e3f
  Stored in directory: /root/.cache/pip/wheels/2e/93/b7/cf649194508e53cee4145ffb949e9f26877a5a8dd12db9ed5b
Successfully built ta


In [9]:
embeddings = get_embeddings(corr_df=df, n_epochs=10, emb_dim=32, silent=True)
tr = preprocessing(data=data, start=1, end=121, corr_df=df)
va = preprocessing(data=data, start=122, end=186, corr_df=df)

tdmin = tr['date_index'].min()
tdmax = tr['date_index'].max()
vdmin = va['date_index'].min()
vdmax = va['date_index'].max()
train = tr[tr['date_index']>=tdmin+24]
valid = va[va['date_index']>=vdmin+24]
tr = train[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')
va = valid[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')

tr.head()

,stock_id,date_index,indicator,indicator_values
0,000020,25,EMA,9127.360590
1,000020,26,EMA,9047.766653
2,000020,27,EMA,8841.956399
3,000020,28,EMA,8717.040030
4,000020,29,EMA,8612.880025


In [62]:
def attention_optimize(train_data=None, valid_data=None, lr=0.02, n_epochs=300, embeddings=None, silent=False):

    tdmin = train_data['date_index'].min()
    tdmax = train_data['date_index'].max()
    vdmin = valid_data['date_index'].min()
    vdmax = valid_data['date_index'].max()
    train = train_data[train_data['date_index']>=tdmin+25]
    valid = valid_data[valid_data['date_index']>=vdmin+25]
    train_data = train[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')
    valid_data = valid[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')

    # print('traindata')
    # print(tdmin)
    # print(train_data)

    dtype = torch.FloatTensor
    weights = Variable(torch.randn(7, 32).type(dtype), requires_grad=True) # 7: num_indicators, 32: dim_embeddings
    optimizer = optim.Adam([weights], lr=lr)
    base_loss = np.Inf

    retur = np.array(train[train['date_index']>=tdmin+26].pivot(index='date_index', columns='stock_id', values='return')) # (days-1) x 1586
    v_retur = np.array(valid[valid['date_index']>=vdmin+26].pivot(index='date_index', columns='stock_id', values='return')) # (days-1) x 1586
    final_factor = None

    for epoch in range(n_epochs):

        weighted_indicator = None
        v_weighted_indicator = None

        C = torch.mm(weights, embeddings) # 7 x 1586
        C = F.softmax(C, dim=0)
        # print('C')
        # print(C)

        for day in range(tdmin+25, tdmax):
            indic = np.array(train_data[train_data['date_index']==day].pivot(index='indicator', columns='stock_id', values='indicator_values')) # 7 x 1586
            # print('indic')
            # print(indic)
            # print('C')
            # print(C)
            # print('-'*50)
            # print('indic')
            # print(torch.from_numpy(indic))
            # print('-'*50)
            # print('mid')
            # print(C*torch.from_numpy(indic))
            rescaled_I = torch.sum(C*torch.from_numpy(indic), axis=0).view(1, 1586) # 1 x 1586, elementwise
            # print('rescaled_I')
            # print(rescaled_I)
            if weighted_indicator == None:
                weighted_indicator = rescaled_I
            else:
                weighted_indicator = torch.cat((weighted_indicator, rescaled_I), dim=0)
        vx = torch.from_numpy(retur) - torch.mean(torch.from_numpy(retur), axis=0) 
        vy = weighted_indicator - torch.mean(weighted_indicator, axis=0) 
        # print('vy')
        # print(vy)
        loss = - torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))

        if not silent:
            if (epoch+1) % 100 == 0:
                print("{0}th epoch in process, loss: {1:.2f}".format(epoch+1, loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():

            # Softmax 했으니까 weight 자체를 bound 할 필요는 없지 않나?
            # for weight in weights:
            #     weight.clamp_(0,1)

            for day in range(vdmin+25, vdmax):
                indic = np.array(valid_data[valid_data['date_index']==day].pivot(index='indicator', columns='stock_id', values='indicator_values')) # 7 x 1586
                rescaled_I = torch.sum(C.detach()*torch.from_numpy(indic), axis=0).view(1, 1586) # 1 x 1586
                if v_weighted_indicator == None:
                    v_weighted_indicator = rescaled_I
                else:
                    v_weighted_indicator = torch.cat((v_weighted_indicator, rescaled_I), dim=0)                    
            ux = torch.from_numpy(v_retur) - torch.mean(torch.from_numpy(v_retur), axis=0)
            uy = v_weighted_indicator - torch.mean(v_weighted_indicator, axis=0)
            # print('uy')
            # print(uy)
            v_loss = (-torch.sum(ux * uy) / (torch.sqrt(torch.sum(ux ** 2)) * torch.sqrt(torch.sum(uy ** 2)))).detach().numpy()
            # print(v_loss, base_loss)
            if v_loss < base_loss:
                base_loss = v_loss
                final_factor = C
    if final_factor == None:
        print('Something is wrong')
    return final_factor

In [85]:
def attention_test(data=None, corr_df=None, start=None, end=None, window=62, embeddings=None, e=5):

    from collections import defaultdict
    ans = pd.DataFrame()
    train_df = preprocessing(data=data, corr_df=corr_df, start=start, end=end)
    valid_df = preprocessing(data=data, corr_df=corr_df, start=end+1, end=end+window)
    test_df = preprocessing(data=data, corr_df=corr_df, start=end+window+1, end=end+2*window)
    tdmin = test_df['date_index'].min()
    tdmax = test_df['date_index'].max()
    test_df = test_df[test_df['date_index']>=tdmin + 25]
    test = test_df[['stock_id', 'date_index', 'EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']].melt(id_vars=['stock_id', 'date_index'], var_name='indicator', value_name='indicator_values')
    indicators=['EMA', 'MACD', 'KLength', 'KUpperLength', 'KLowerLength', 'ROC', 'BIAS']
    result = defaultdict(list)
    retur = np.array(test_df[test_df['date_index']>=tdmin+26].pivot(index='date_index', columns='stock_id', values='return')) # (days-1) x 1586
    ux = torch.from_numpy(retur) - torch.mean(torch.from_numpy(retur), axis=0) # (days-1) x 1586
    for indica in indicators:
        f = np.array(test_df[(test_df['date_index']>=tdmin+25)&(test_df['date_index']<tdmax)].pivot(index='date_index', columns='stock_id', values=indica)) # (days-1) x 1586
        uz = torch.from_numpy(f).float() - torch.mean(torch.from_numpy(f).float(), axis=0)
        ans[indica] = (torch.sum(ux * uz, axis=0) / (torch.sqrt(torch.sum(ux ** 2, axis=0)) * torch.sqrt(torch.sum(uz ** 2, axis=0)))).numpy()
    j = torch.zeros(tdmax-tdmin, len(corr_df))

    for _ in range(e):
        C = attention_optimize(train_data=train_df, valid_data=valid_df, n_epochs=500, embeddings=embeddings, silent=True).detach() # set epochs appropriately
        v_weighted_indicator = None
        for day in range(tdmin+25, tdmax):
            indic = np.array(test[test['date_index']==day].pivot(index='indicator', columns='stock_id', values='indicator_values')) # 7 x 1586
            rescaled_I = torch.sum(C.detach()*torch.from_numpy(indic), axis=0).view(1, 1586) # 1 x 1586
            if v_weighted_indicator == None:
                v_weighted_indicator = rescaled_I
            else:
                v_weighted_indicator = torch.cat((v_weighted_indicator, rescaled_I), dim=0)
        uy = v_weighted_indicator - torch.mean(v_weighted_indicator, axis=0) # (days-1) x 1586
        j += (torch.sum(ux * uy, axis=0) / (torch.sqrt(torch.sum(ux ** 2, axis=0)) * torch.sqrt(torch.sum(uy ** 2, axis=0)))).numpy()
    o = torch.mean(j/e, axis=0)
    ans['Weighted'] = o.numpy()
    return ans

In [86]:
# df, embeddings, st, ed = data_preparer(n=1, n_epochs=100, lr=0.05, silent=True, corr=True)
output_df = attention_test(data=data, corr_df=df, start=st, end=ed, embeddings=embeddings)
output_df.index = df.columns
output_df.head()

,EMA,MACD,KLength,KUpperLength,KLowerLength,ROC,BIAS,Weighted
0,-0.377011,-0.159677,-0.314046,0.150986,0.425001,-0.221035,-0.088396,-0.095207
1,-0.280478,0.236013,-0.128294,0.031322,0.160225,-0.077343,-0.151924,0.002224
2,-0.278527,0.031770,-0.184338,-0.013554,0.265729,-0.247099,-0.026860,-0.157970
3,-0.127501,-0.096512,-0.383323,0.157428,0.042104,-0.267488,-0.247537,-0.181723
4,-0.308098,0.002291,-0.114454,-0.001540,0.163401,-0.177281,-0.257411,-0.161698


In [87]:
output_df.index = df.columns
output_df

,EMA,MACD,KLength,KUpperLength,KLowerLength,ROC,BIAS,Weighted
000020,-0.377011,-0.159677,-0.314046,0.150986,0.425001,-0.221035,-0.088396,-0.095207
000060,-0.280478,0.236013,-0.128294,0.031322,0.160225,-0.077343,-0.151924,0.002224
000070,-0.278527,0.031770,-0.184338,-0.013554,0.265729,-0.247099,-0.026860,-0.157970
000075,-0.127501,-0.096512,-0.383323,0.157428,0.042104,-0.267488,-0.247537,-0.181723
000080,-0.308098,0.002291,-0.114454,-0.001540,0.163401,-0.177281,-0.257411,-0.161698
...,...,...,...,...,...,...,...,...
99220,-0.509255,0.407446,-0.113407,0.171789,-0.037614,-0.046879,-0.150960,-0.016472
99320,-0.441376,-0.119497,0.194643,0.022918,-0.030469,-0.239901,0.041634,-0.116764
99410,-0.216342,0.122138,-0.243672,-0.141515,0.110695,-0.062621,-0.125617,0.000184
99440,-0.379924,-0.393725,-0.191283,-0.022740,0.254288,-0.166922,-0.198363,-0.092198


In [89]:
output_df['raw_average'] = output_df.drop(columns=['Weighted']).mean(axis=1)
output_df

,EMA,MACD,KLength,KUpperLength,KLowerLength,ROC,BIAS,Weighted,raw_average
000020,-0.377011,-0.159677,-0.314046,0.150986,0.425001,-0.221035,-0.088396,-0.095207,-0.083454
000060,-0.280478,0.236013,-0.128294,0.031322,0.160225,-0.077343,-0.151924,0.002224,-0.030069
000070,-0.278527,0.031770,-0.184338,-0.013554,0.265729,-0.247099,-0.026860,-0.157970,-0.064697
000075,-0.127501,-0.096512,-0.383323,0.157428,0.042104,-0.267488,-0.247537,-0.181723,-0.131833
000080,-0.308098,0.002291,-0.114454,-0.001540,0.163401,-0.177281,-0.257411,-0.161698,-0.099013
...,...,...,...,...,...,...,...,...,...
99220,-0.509255,0.407446,-0.113407,0.171789,-0.037614,-0.046879,-0.150960,-0.016472,-0.039840
99320,-0.441376,-0.119497,0.194643,0.022918,-0.030469,-0.239901,0.041634,-0.116764,-0.081721
99410,-0.216342,0.122138,-0.243672,-0.141515,0.110695,-0.062621,-0.125617,0.000184,-0.079562
99440,-0.379924,-0.393725,-0.191283,-0.022740,0.254288,-0.166922,-0.198363,-0.092198,-0.156953


In [90]:
output_df['Weighted'].mean(), output_df['raw_average'].mean()

(-0.07683103345929887, -0.07897321661404509)

In [91]:
output_df.to_csv('attention_corr_1.csv', index=False)